In [ ]:
import geopandas as gpd
import os
import requests
import rasterio
from shapely.geometry import Polygon

### This tool will download a DEM that matches the spatial extent of a provided shapefile or raster.

#### Data Location and folder structure:

In [ ]:
DataFolder = input('Input (existing) folder path for Data folder structure:')   # example: 'DATA'

os.chdir(DataFolder)
dirList = ['DEM']
for x in dirList:
    if not os.path.isdir(x):
        os.mkdir(x)

#### File input:

In [ ]:
InputData = input('Input file path:')   # example: _SAMPLE/03_OregonDams_shp.shp

#### Definitions:

In [ ]:
def makeGDF(shape_path):
    shape_gdf = gpd.read_file(shape_path)
    return shape_gdf



def makeExtentPoly(raster_path):
    
    raster = rasterio.open(raster_path)
    raster_name = os.path.splitext(raster_path)[0].split('/')[1]
    
    
    left = raster.bounds[0] 
    lower = raster.bounds[1]
    right = raster.bounds[2]
    upper = raster.bounds[3]

    # Set raster bounding box
    raster_box = [[left, upper], [right, upper], [right, lower], [left, lower]]

    # define the geometry
    raster_poly = Polygon(raster_box)
    d = {'name':[f'{raster_name}',], 'geometry': [raster_poly,]}

    # make the geodataframe
    raster_extent = gpd.GeoDataFrame(d, crs=str(raster.crs))
    
    return raster_extent


def makeExtent(InputData):

    if InputData.endswith('.shp'):
        extent = makeGDF(InputData)

    elif InputData.endswith('.tif'):
        extent = makeExtentPoly(InputData)
    else:
        print("Data type not known. Must be a .shp or .tif")
    return extent


def Download_DEM(geodataframe):
    

    xMin = round(geodataframe.total_bounds[0], 3)
    yMin = round(geodataframe.total_bounds[1], 3)
    xMax = round(geodataframe.total_bounds[2], 3)
    yMax = round(geodataframe.total_bounds[3], 3)
    boundingBox = "{},{},{},{}".format(xMin, yMin, xMax, yMax)


    linkStart = "https://tnmaccess.nationalmap.gov/api/v1/products?datasets=National%20Elevation%20Dataset%20(NED)%201/3%20arc-second&"
    boundingBox_forLink = "bbox=" + str(boundingBox)
    linkFinish = "&prodFormats=GeoTIFF&outputFormat=JSON"
    TNM_Link = linkStart + boundingBox_forLink + linkFinish

    r = requests.get(TNM_Link)
    json_data = r.json()
    downloadList = []

    for item in json_data["items"]:
        downloadList.append(item["downloadURL"])

    fileCounter = 1
    RastersList = []

    for downURL in downloadList:
        fileSplit = downURL.split("/")
        fileName = fileSplit[-1]

        filePath = os.path.join('DEM', fileName)

        RastersList.append(filePath)

        print (f'Downloading {fileName} file {fileCounter} of {len(downloadList)}')

        fileDown = requests.get(downURL)

        with open(filePath, 'wb') as asdf:
            asdf.write(fileDown.content)
        fileCounter += 1
        
        print ('Finished downloading')
    
    return RastersList





#### Run to download DEM:

In [ ]:
extent = makeExtent(InputData)


In [ ]:
Download_DEM(extent)